In [12]:

#Import checks
from pip._internal import main as pipmain

try:
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('vader_lexicon')
except:

    pipmain(["install", "nltk"])
    nltk.download('stopwords')
    import nltk
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import PorterStemmer
    nltk.download('stopwords')
try:
    import re
except:
    pipmain(["install", "re"])
    import re
try:
    import string
except:
    pipmain(["install", "string"])
    import string
try:
    import unicodedata
except:
    pipmain(["install", "unicodedata"])
    import unicodedata
try:
    import pandas
except:
    pipmain(["install", "pandas"])
    import pandas
try:
    import os
except:
    pipmain(["install", "os"])
    import os
try:
    import collections
except:
    pipmain(["install", "collections"])
    import collections
try:
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
except:
    pipmain(["install", "sklearn"])
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
try:
    import tweepy
except:
    pipmain(["install", "tweepy"])
    import tweepy
try:
    import csv
except:
    pipmain(["install", "csv"])
    import csv
try:
    import time
except:
    pipmain(["install", "time"])
    import time
try:
    import pickle
except:
    pipmain(["install", "pickle"])
    import pickle


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Nick\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
import unicodedata
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') # or any other list of your choice
def TweetbScrub(text, ):
    if text[0] == "b":
        text = text[1:]
    return text

def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

    def remove_special_characters(text, characters = string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = TweetbScrub(text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'[^^a-zA-Z ]', '', text)
    text = text.strip(' ') # strip whitespaces
    text = text.lower() # lowercase
    text = stem_text(text) # stemming
    try:
        text = remove_special_characters(text) # remove punctuation and symbols
    except:
        print("Error")
    text = remove_stopwords(text) # remove stopwords
    #text.strip(' ') # strip whitespaces again?

    return text

PartyColours = {"Labour": "#dc241f", 
              "Conservative": "#0087dc", 
              "SNP": "#fff95d", 
              "Green": "#6ab023", 
              "UKIP": "#70147a", 
              "DUP":"#d46a4c", 
              "Non-Aligned": "#ffffff", 
              "LibDem": "#fdbb30"}

#Twitter API credentials
consumer_key = "TC98w89JxQK2v4vPEqLLxJLx0"
consumer_secret = "le4t2JCgoT3CBZwToaKdOJx5LFYDDkGL5e3Pjl2ZtfTqYV46Fs"
access_key = "4459846396-tU9aYf4E5r9eHhJnniU7OsyrLNJhzEd4cpVeFFe"
access_secret = "UaY6kpdXbdV7cAsAxrKLzFTkKSLtW8dcNTe1CYniUl6xM"



def get_all_tweets(screen_name):
    
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv



    outtweets = []
    for tweet in alltweets:
        media_count = 0
        url_count =0
        if "media" in tweet.entities:
            media_count =1
        if "urls" in tweet.entities:
            url_count =len(tweet.entities["urls"])
        outtweets.append([tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"),tweet.retweet_count,tweet.favorite_count,media_count,url_count])

    #write the csv
    with open(screen_name + '.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text","retweet_count","favorite_count","media_count","url_count"])
        writer.writerows(outtweets)

    pass

def get_all_favourites(screen_name):
    
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.favorites(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.favorites(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv



    outtweets = []
    for tweet in alltweets:
        media_count = 0
        url_count =0
        if "media" in tweet.entities:
            media_count =1
        if "urls" in tweet.entities:
            url_count =len(tweet.entities["urls"])
        outtweets.append([tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"),tweet.retweet_count,tweet.favorite_count,media_count,url_count])

    #write the csv
    with open(screen_name + '.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text","retweet_count","favorite_count","media_count","url_count"])
        writer.writerows(outtweets)

    pass

#Remove Retweets
def Smallclean(text, ):
    text = TweetbScrub(text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'[^^a-zA-Z@1-9 ]', '', text)
    return text

def FindTop(UserData):
    
    UserData = UserData[~UserData.text.str.contains("RT ")]
    favourite = UserData.loc[UserData['favorite_count'].idxmax()]
    retweet = UserData.loc[UserData['retweet_count'].idxmax()]
    file = open("Favourite.txt","w") 
    file.writelines(favourite['text'] + "\n")
    file.writelines(str(favourite['favorite_count']))
    file.close()

    file = open("Retweet.txt","w") 
    file.writelines(retweet['text'] + "\n")
    file.writelines(str(retweet['retweet_count']))
    file.close()

def GeneratePie(collection, name):
    import plotly 
    plotly.tools.set_credentials_file(username='itjallingii', api_key='hiFqLEhfwSwdDlsj9lT8')

    import plotly.plotly as py
    import plotly.graph_objs as go


    labels = list(collection.keys())
    values = list(collection.values())
    colors = [PartyColours[i] for i in labels]


    trace = go.Pie(labels=labels, values=values,
                   hoverinfo='label+percent', textinfo='value', 
                   textfont=dict(size=10),
                   marker=dict(colors = colors,
                               line=dict(color='#000000', width=2)))

    data = [trace]
    layout = go.Layout(title=name)
    fig = go.Figure(data=data,layout=layout)

    # Save the figure as a png image:
    py.image.save_as(fig, name + '.png')

def GeneratePieS(collection, name):
    import plotly 
    plotly.tools.set_credentials_file(username='itjallingii', api_key='hiFqLEhfwSwdDlsj9lT8')

    import plotly.plotly as py
    import plotly.graph_objs as go


    labels = list(collection.keys())
    values = list(collection.values())
    #colors = [PartyColours[i] for i in labels]


    trace = go.Pie(labels=labels, values=values,
                   hoverinfo='label+percent', textinfo='value', 
                   textfont=dict(size=10),
                   marker=dict(
                               line=dict(color='#000000', width=2)))

    data = [trace]
    layout = go.Layout(title=name)
    fig = go.Figure(data=data,layout=layout)

    # Save the figure as a png image:
    py.image.save_as(fig, name + '.png')

def AnalyseSpeech(Filename, Mode):
    os.chdir("E:\Honours\Master\Speeches")
    Filename = Filename.replace(".txt", "")
    Speech = open(Filename + ".txt").read()
    Sentence = vectorizer.transform([Speech])
    classifier.predict(Sentence)
    Speech = Speech.replace("\n", "")
    Speech = Speech.split(".")
    for sentence in Speech:
        if sentence == " ":
            Speech.remove(sentence)
    Speech = list(filter(None, Speech))

    Speech = pandas.DataFrame(Speech)
    Speech["Classification"] = ""
    Speech.columns = ['Text', "Classification"]
    Speech['CleanText'] = Speech['Text'].apply(lambda row: clean_text(row))
    Sentence = vectorizer.transform(Speech["CleanText"])
    Classification = classifier.predict(Sentence)
    Speech["Classification"] = Classification
    SpeechS = Speech
    if Mode == "British":
        SpeechS = Speech.apply(SentimentclassiferBritish,axis=1)
    else:
        SpeechS = Speech.apply(SentimentclassiferAmerican,axis=1)
    Speech.drop('CleanText', axis=1, inplace=True)
    SpeechS.drop('CleanText', axis=1, inplace=True)
    Speechcollection = collections.Counter(Speech["Classification"])
    SpeechScollection = collections.Counter(SpeechS["Classification"])
    #Change directory to the classifications directory for saving
    if Mode == "British":
        os.chdir("E:\Honours\Master\Speeches\Classifications\British")
    else:
        os.chdir("E:\Honours\Master\Speeches\Classifications\American")
    #Save Classifications to classificaiton directory
    try:
        os.mkdir(Filename)
    except:
        pass
    os.chdir(Filename)
    Speech.to_csv(Filename + "-Classification.csv", encoding='utf-8')
    GeneratePie(Speechcollection, Filename)
    Speech.to_csv(Filename + "-Sentiment-Classification.csv", encoding='utf-8')
    if Mode == "British":
        GeneratePieS(SpeechScollection, Filename + "-Sentiment")
    else:
        GeneratePie(SpeechScollection, Filename + "-Sentiment")

def AnalyseTweets(Username, Mode):
    os.chdir("E:\Honours\Master\Twitter-User-Data\Classifications")
    try:
        os.mkdir(Username)
    except:
        pass
    os.chdir(Username)

    get_all_tweets(Username)
    Username = Username.replace(".csv", "")
    UserData = pandas.DataFrame.from_csv(Username + ".csv")
    FindTop(UserData)
    #Vectorize the text
    
    UserData['CleanText'] = UserData['text'].apply(lambda row: clean_text(row))
    Sentences = vectorizer.transform(UserData['CleanText'])
    #Perform prediction of text inputted
    Classification = classifier.predict(Sentences)
    #Output a basic overview of the predictions
    #Append original data with model classification
    UserData["Classification"] = Classification
    UserDataSentiment = UserData
    
    Usercollection = collections.Counter(UserData["Classification"])
    w = csv.writer(open("output.csv", "w"))
    for key, val in Usercollection.items():
            w.writerow([key, val])
    UserData.to_csv(Username + "-Tweets-Classification.csv", encoding='utf-8')
    GeneratePie(Usercollection, Username + "-Tweets")

    if Mode == "British":
        UserDataSentiment = UserDataSentiment.apply(SentimentclassiferBritish,axis=1)
    else:
        UserDataSentiment = UserDataSentiment.apply(SentimentclassiferAmerican,axis=1)
    UserData.drop('CleanText', axis=1, inplace=True)
    UserDataSentiment.drop('CleanText', axis=1, inplace=True)
    UsercollectionSentiment = collections.Counter(UserDataSentiment["Classification"])
    #Change directory to the classifications directory for saving
    #Save Classifications to classificaiton directory
    
    UserData.to_csv(Username + "-Tweets-Classification.csv", encoding='utf-8')
    GeneratePie(Usercollection, Username + "-Tweets")
    UserDataSentiment.to_csv(Username + "-Tweets-Classification-S.csv", encoding='utf-8')
    if Mode == "British":
        GeneratePieS(UsercollectionSentiment, Username + "-Tweets-S")
    else:
        GeneratePie(UsercollectionSentiment, Username + "-Tweets-S")

    os.remove(Username + ".csv")

    
def AnalyseFavourites(Username, Mode):
    os.chdir("E:\Honours\Master\Twitter-User-Data\Classifications")
    try:
        os.mkdir(Username)
    except:
        pass
    os.chdir(Username)
    get_all_favourites(Username)
    Username = Username.replace(".csv", "")
    UserData = pandas.DataFrame.from_csv(Username + ".csv")
    #Vectorize the text
    
    UserData['CleanText'] = UserData['text'].apply(lambda row: clean_text(row))
    Sentences = vectorizer.transform(UserData['CleanText'])
    #Perform prediction of text inputted
    Classification = classifier.predict(Sentences)
    #Output a basic overview of the predictions
    #Append original data with model classification
    UserData["Classification"] = Classification
    UserDataSentiment = UserData


    Usercollection = collections.Counter(UserData["Classification"])
    #Change directory to the classifications directory for saving
    #Save Classifications to classificaiton directory
    
    if Mode == "British":
        UserDataSentiment = UserDataSentiment.apply(SentimentclassiferBritish,axis=1)
    else:
        UserDataSentiment = UserDataSentiment.apply(SentimentclassiferAmerican,axis=1)
    
    UserData.drop('CleanText', axis=1, inplace=True)
    UserDataSentiment.drop('CleanText', axis=1, inplace=True)
    UsercollectionSentiment = collections.Counter(UserDataSentiment["Classification"])
    #Change directory to the classifications directory for saving
    #Save Classifications to classificaiton directory
    UserDataSentiment.to_csv(Username + "-Favourites-Classification-S.csv", encoding='utf-8')
    if Mode == "British":
        GeneratePieS(UsercollectionSentiment, Username + "-Favourites-S")
    else:
        GeneratePie(UsercollectionSentiment, Username + "-Favourites-S")
    UserData.to_csv(Username + "-Favourites-Classification.csv", encoding='utf-8')
    GeneratePie(Usercollection, Username + "-Favourites")
    os.remove(Username + ".csv")
    
def AnalyseUser(Username, Mode):
    AnalyseTweets(Username, Mode)
    AnalyseFavourites(Username, Mode)

    
    
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def SentimentclassiferBritish(sentence, ):
    sentiment = sid.polarity_scores(sentence['CleanText'])
    sentence['Sentiment'] = sentiment['compound']
    if (sentence['Sentiment'] < -0) & (sentence['Classification'] != "Non-Aligned"):
        sentence['Classification'] = "Anti-" + sentence['Classification']
    return sentence
def SentimentclassiferAmerican(sentence, ):
    sentiment = sid.polarity_scores(sentence['CleanText'])
    sentence['Sentiment'] = sentiment['compound']
    if (sentence['Sentiment'] < -0) & (sentence['Classification'] == "Republican"):
        sentence['Classification'] = "Democratic"
    elif (sentence['Sentiment'] < -0) & (sentence['Classification'] == "Democratic"):
        sentence['Classification'] = "Republican"
    return sentence

In [14]:
#Load British
def LoadBritish():
    global Mode 
    global vectorizer
    global classifier
    global PartyColours 
    Mode = "British"
    print("Loading Mode: " + Mode) 
    PartyColours = {"Labour": "#dc241f", 
                  "Conservative": "#0087dc", 
                  "SNP": "#fff95d", 
                  "Green": "#6ab023", 
                  "UKIP": "#70147a", 
                  "DUP":"#d46a4c", 
                  "Non-Aligned": "#ffffff", 
                  "LibDem": "#fdbb30"}
    try:
        os.chdir("E:\Honours\Master\Models")
        print("File location found")
        classifier = pickle.load(open("ModelB", 'rb'))
        print("Pickle Model Loaded")
        BritishDF = pandas.DataFrame.from_csv("BritishCleaned.csv")
        print("CSV loaded")
        vectorizer = CountVectorizer(min_df=0, lowercase=False)
        print("Vector built")
        BritishDF['text'] = BritishDF['text'].values.astype('U')
        vectorizer.fit(BritishDF['text'].values.astype('U'))
        print("Data fit")
        sentences = BritishDF['text'].values
        y = BritishDF['Alignment'].values

        sentences_train, sentences_test, y_train, y_test = train_test_split(
            sentences, y, test_size=0.05, random_state=1000)

        vectorizer = CountVectorizer()
        vectorizer.fit(sentences_train)

        X_train = vectorizer.transform(sentences_train)
        X_test  = vectorizer.transform(sentences_test)

        score = classifier.score(X_test, y_test)

    except:
        print("Error")
        os.chdir("E:\Honours\Master\Twitter-Learning-Data\Collections")

        BritishDF = pandas.DataFrame.from_csv("AmericanCollection.csv")

        BritishDF['text'] = BritishDF['text'].apply(lambda row: clean_text(row))

        vectorizer = CountVectorizer(min_df=0, lowercase=False)

        vectorizer.fit(BritishDF['text'])

        sentences = BritishDF['text'].values
        y = BritishDF['Alignment'].values

        sentences_train, sentences_test, y_train, y_test = train_test_split(
            sentences, y, test_size=0.05, random_state=1000)

        vectorizer = CountVectorizer()
        vectorizer.fit(sentences_train)

        X_train = vectorizer.transform(sentences_train)
        X_test  = vectorizer.transform(sentences_test)

        classifier = LogisticRegression()
        classifier.fit(X_train, y_train)
        score = classifier.score(X_test, y_test)

        print("Accuracy:", score)

        os.chdir("E:\Honours\Master\Models")
        pickle.dump(classifier, open("ModelB", 'wb'))
        BritishDF.to_csv("BritishCleaned.csv", encoding='utf-8')

In [15]:
#Load American
def LoadAmerican():
    global Mode 
    global vectorizer
    global classifier
    global PartyColours 
    Mode = "American"
    print("Loading Mode: " + Mode) 
    PartyColours = {"Republican": "#ff0000", 
                  "Democratic": "#0015BC", 
                  "Non-Aligned": "#ffffff"}
    try:
        os.chdir("E:\Honours\Master\Models")
        print("File location found")
        classifier = pickle.load(open("ModelA", 'rb'))
        print("Pickle Model Loaded")
        AmericanDF = pandas.DataFrame.from_csv("AmericanCleaned.csv")
        print("CSV loaded")
        vectorizer = CountVectorizer(min_df=0, lowercase=False)
        print("Vector built")
        AmericanDF['text'] = AmericanDF['text'].values.astype('U')
        vectorizer.fit(AmericanDF['text'].values.astype('U'))
        print("Data fit")
        sentences = AmericanDF['text'].values
        y = AmericanDF['Alignment'].values

        sentences_train, sentences_test, y_train, y_test = train_test_split(
            sentences, y, test_size=0.05, random_state=1000)

        vectorizer = CountVectorizer()
        vectorizer.fit(sentences_train)

        X_train = vectorizer.transform(sentences_train)
        X_test  = vectorizer.transform(sentences_test)

        score = classifier.score(X_test, y_test)

    except:
        print("Error")
        os.chdir("E:\Honours\Master\Twitter-Learning-Data\Collections")

        AmericanDF = pandas.DataFrame.from_csv("AmericanCollection.csv")

        AmericanDF['text'] = AmericanDF['text'].apply(lambda row: clean_text(row))

        vectorizer = CountVectorizer(min_df=0, lowercase=False)

        vectorizer.fit(AmericanDF['text'])

        sentences = AmericanDF['text'].values
        y = AmericanDF['Alignment'].values

        sentences_train, sentences_test, y_train, y_test = train_test_split(
            sentences, y, test_size=0.05, random_state=1000)

        vectorizer = CountVectorizer()
        vectorizer.fit(sentences_train)

        X_train = vectorizer.transform(sentences_train)
        X_test  = vectorizer.transform(sentences_test)

        classifier = LogisticRegression()
        classifier.fit(X_train, y_train)
        score = classifier.score(X_test, y_test)

        print("Accuracy:", score)

        os.chdir("E:\Honours\Master\Models")
        pickle.dump(classifier, open("ModelA", 'wb'))
        AmericanDF.to_csv("AmericanCleaned.csv", encoding='utf-8')


In [ ]:
os.chdir("E:\Honours\Master\Twitter-User-Data\Raw")

for filename in os.listdir():
        if ".csv" in filename: 
            AnalyseUser(filename)
            


In [ ]:
os.chdir("E:\Honours\Master\Speeches")
for filename in os.listdir():
        if ".txt" in filename: 
            AnalyseSpeech(filename)
            
        


In [16]:
#Analyse user test
LoadBritish()
AnalyseUser("wildfireone", Mode)

Loading Mode: British
Error


WindowsError: [Error 3] The system cannot find the path specified: 'E:\\Honours\\Master\\Twitter-Learning-Data\\Collections'

In [ ]:
#AnalyseSpeech Test
AnalyseSpeech("WordList")

In [5]:
#MultiMode Test
LoadAmerican()
AnalyseSpeech("Hitler-Danzig", Mode)

LoadBritish()
AnalyseSpeech("Hitler-Danzig", Mode)

American
File location found
Pickle Model Loaded


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


CSV loaded
Vector built
Data fit
American
File location found
Pickle Model Loaded


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



CSV loaded
Vector built
Data fit
British


In [6]:
#Single word test
x = vectorizer.transform(["Indepedence!"])
classifier.predict(x)

array(['Non-Aligned'], dtype=object)

In [14]:
#Find Data
Username = "ZoeJP96"
os.chdir("E:\Honours\Master\Twitter-User-Data\Classifications")
try:
    os.mkdir(Username)
except:
    pass
os.chdir(Username)

Username = Username.replace(".csv", "")
UserData = pandas.DataFrame.from_csv(Username + ".csv")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # This is added back by InteractiveShellApp.init_path()


In [15]:
UserData['cleantext'] = UserData['text'].apply(lambda row: clean_text(row))
UserData['cleantext']

id
1107390126987059200                            agirlcalledlina ellienich
1105981012310482946    rt theglasgowcup xfxfxfxxfxfxfxbcxexxdxexxxefx...
1105519018822242311                                               bradyy
1105518834943954944    rt bradyy creas zoejp got told say hi silent f...
1105518566630146049    rt bradyy minut spare pleas fill thi survey he...
1104793018379526145    elouisee pryceconni samanthahawk everi hall pa...
1102275664177233922                              oldi xfxfxxe pure steam
1102150607199260673                         bedroomflick ellienich savag
1101490205209575424    rosskylejohnson wehden supposedli pay xcxa ana...
1101254830876737536                           hedgehogclown twitter user
1101175538947948544    rt jimwaterson coverag momo challeng possibl i...
1100844860473528321    xfxfxaxa ixexxm current write honour dissert m...
1100128616594239490    rt fkajack xexxci boyfriend told ixexxd never ...
1099648712320651270                             

"b'RT @carterjwm: HELP ME PLEASE. A MAN NEEDS HIS NUGGS https://t.co/4SrfHmEMo3'"

In [108]:
#Remove Retweets
def Smallclean(text, ):
    text = TweetbScrub(text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'[^^a-zA-Z@1-9 ]', '', text)
    return text

def FindTop(UserData):
    
    UserData = UserData[~UserData.text.str.contains("RT ")]
    favourite = UserData.loc[UserData['favorite_count'].idxmax()]
    retweet = UserData.loc[UserData['retweet_count'].idxmax()]
    file = open("Favourite.txt","w") 
    file.writelines(favourite['text'] + "\n")
    file.writelines(str(favourite['favorite_count']))
    file.close()

    file = open("Retweet.txt","w") 
    file.writelines(retweet['text'] + "\n")
    file.writelines(str(retweet['retweet_count']))
    file.close()

In [101]:
file1 = open("Favourite.txt","w") 
file1.writelines(favourite['text'] + "\n")
file1.writelines(str(favourite['favorite_count']))
file1.close()
                 
file1 = open("Retweet.txt","w") 
file1.writelines(retweet['text'] + "\n")
file1.writelines(str(retweet['retweet_count']))
file1.close()

In [34]:
file1 = open("Favourite.txt","a") 
tweet['text'] = TweetbScrub(tweet['text'])
file1.write(retweet['text'])
file1.close()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
Username = "wildfireone"
os.chdir("E:\Honours\Master\Twitter-User-Data\Classifications")
try:
    os.mkdir(Username)
except:
    pass
os.chdir(Username)

#get_all_tweets(Username)
Username = Username.replace(".csv", "")
UserData = pandas.DataFrame.from_csv(Username + ".csv")
FindTop(UserData)
#Vectorize the text

UserData['CleanText'] = UserData['text'].apply(lambda row: clean_text(row))
Sentences = vectorizer.transform(UserData['CleanText'])
#Perform prediction of text inputted
Classification = classifier.predict(Sentences)
#Output a basic overview of the predictions
#Append original data with model classification
UserData["Classification"] = Classification
UserDataSentiment = UserData

Usercollection = collections.Counter(UserData["Classification"])
w = csv.writer(open("output.csv", "w"))
for key, val in Usercollection.items():
        w.writerow([key, val])
UserData.to_csv(Username + "-Tweets-Classification.csv", encoding='utf-8')
GeneratePie(Usercollection, Username + "-Tweets")

if Mode == "British":
    UserDataSentiment = UserDataSentiment.apply(SentimentclassiferBritish,axis=1)
else:
    UserDataSentiment = UserDataSentiment.apply(SentimentclassiferAmerican,axis=1)
UserData.drop('CleanText', axis=1, inplace=True)
UserDataSentiment.drop('CleanText', axis=1, inplace=True)
UsercollectionSentiment = collections.Counter(UserDataSentiment["Classification"])
#Change directory to the classifications directory for saving
#Save Classifications to classificaiton directory

UserData.to_csv(Username + "-Tweets-Classification.csv", encoding='utf-8')
GeneratePie(Usercollection, Username + "-Tweets")
UserDataSentiment.to_csv(Username + "-Tweets-Classification-S.csv", encoding='utf-8')
if Mode == "British":
    GeneratePieS(UsercollectionSentiment, Username + "-Tweets-S")
else:
    GeneratePie(UsercollectionSentiment, Username + "-Tweets-S")

os.remove(Username + ".csv")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # This is added back by InteractiveShellApp.init_path()


ModuleNotFoundError: No module named 'plotly'

In [23]:
Usercollection

Counter({'Non-Aligned': 1936,
         'SNP': 512,
         'Conservative': 157,
         'Labour': 226,
         'UKIP': 63,
         'LibDem': 137,
         'DUP': 30,
         'Green': 136})

In [24]:
w = csv.writer(open("output.csv", "w"))
for key, val in Usercollection.items():
        w.writerow([key, val])